In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import utils
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPool2D, Input, Dense, Flatten, Concatenate
from keras.callbacks import ModelCheckpoint

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
from IPython.display import Image

import os
import random
from copy import deepcopy
import pickle

In [2]:
print(tf.__version__)

2.8.0


In [3]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
# from tensorflow.keras.applications.resnet import ResNet101
# from tensorflow.keras.applications.resnet150 import ResNet150
from tensorflow.keras.applications.resnet50 import ResNet50

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.efficientnet import EfficientNetB0

In [4]:
fashion_mnist = keras.datasets.fashion_mnist
((x_train, y_train), (x_test, y_test)) = fashion_mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000, 10)
(10000, 28, 28)
(10000, 10)


In [4]:
IMG_SHAPE = (32, 32, 3)
resnet50 = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
len(resnet50.layers)


175

In [8]:
resnet50.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 38, 38, 3)    0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 16, 16, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 16, 16, 64)   256         ['conv1_conv[0][0]']             
                                                                                           

In [5]:
from keras.utils.vis_utils import plot_model
# plot model architecture
plot_model(resnet50, show_shapes=True, to_file='resnet50.png')

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [6]:
resnet50.trainable = True		# resnet 모델 학습동결을 해제한다
# for idx, i in enumerate(resnet50.layers):	# 143층부터의 학습은 해제상태로 두고, 
#   i.trainable = bool_arr[idx]				# 이전까지의 학습은 동결한다.

idx = 0 
for i in resnet50.layers[:]:	# 동결이 제대로 해제됐는지 약간 이전층부터 출력해본다.
  if i.name[:12] == 'conv2_block1':
    print(idx, 'layer :  ', i.name)
  elif i.name[:12] == 'conv2_block2':
    print(idx, 'layer :  ', i.name)
  elif i.name[:12] == 'conv2_block3':
    print(idx, 'layer :  ', i.name)

  elif i.name[:12] == 'conv3_block1':
    print(idx, 'layer :  ', i.name)
  elif i.name[:12] == 'conv3_block2':
    print(idx, 'layer :  ', i.name)
  elif i.name[:12] == 'conv3_block3':
    print(idx, 'layer :  ', i.name)
  elif i.name[:12] == 'conv3_block4':
    print(idx, 'layer :  ', i.name)

  elif i.name[:12] == 'conv4_block1':
    print(idx, 'layer :  ', i.name)
  elif i.name[:12] == 'conv4_block2':
    print(idx, 'layer :  ', i.name)
  elif i.name[:12] == 'conv4_block3':
    print(idx, 'layer :  ', i.name)
  elif i.name[:12] == 'conv4_block4':
    print(idx, 'layer :  ', i.name)
  elif i.name[:12] == 'conv4_block5':
    print(idx, 'layer :  ', i.name)
  elif i.name[:12] == 'conv4_block6':
    print(idx, 'layer :  ', i.name)

  elif i.name[:12] == 'conv5_block1':
    print(idx, 'layer :  ', i.name)
  elif i.name[:12] == 'conv5_block2':
    print(idx, 'layer :  ', i.name)
  elif i.name[:12] == 'conv5_block3':
    print(idx, 'layer :  ', i.name)
    
  idx += 1
  
'''
conv2_block1
conv2_block2
conv2_block3

conv3_block1
conv3_block2
conv3_block3
conv3_block4

conv4_block1
conv4_block2
conv4_block3
conv4_block4
conv4_block5
conv4_block6

conv5_block1
conv5_block2
conv5_block3
'''

7 layer :   conv2_block1_1_conv
8 layer :   conv2_block1_1_bn
9 layer :   conv2_block1_1_relu
10 layer :   conv2_block1_2_conv
11 layer :   conv2_block1_2_bn
12 layer :   conv2_block1_2_relu
13 layer :   conv2_block1_0_conv
14 layer :   conv2_block1_3_conv
15 layer :   conv2_block1_0_bn
16 layer :   conv2_block1_3_bn
17 layer :   conv2_block1_add
18 layer :   conv2_block1_out
19 layer :   conv2_block2_1_conv
20 layer :   conv2_block2_1_bn
21 layer :   conv2_block2_1_relu
22 layer :   conv2_block2_2_conv
23 layer :   conv2_block2_2_bn
24 layer :   conv2_block2_2_relu
25 layer :   conv2_block2_3_conv
26 layer :   conv2_block2_3_bn
27 layer :   conv2_block2_add
28 layer :   conv2_block2_out
29 layer :   conv2_block3_1_conv
30 layer :   conv2_block3_1_bn
31 layer :   conv2_block3_1_relu
32 layer :   conv2_block3_2_conv
33 layer :   conv2_block3_2_bn
34 layer :   conv2_block3_2_relu
35 layer :   conv2_block3_3_conv
36 layer :   conv2_block3_3_bn
37 layer :   conv2_block3_add
38 layer :   co

'\nconv2_block1\nconv2_block2\nconv2_block3\n\nconv3_block1\nconv3_block2\nconv3_block3\nconv3_block4\n\nconv4_block1\nconv4_block2\nconv4_block3\nconv4_block4\nconv4_block5\nconv4_block6\n\nconv5_block1\nconv5_block2\nconv5_block3\n'

In [6]:
IMG_SHAPE = (32, 32, 3)
base_model = MobileNet(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
len(base_model.layers)

86

In [11]:
base_model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv1 (Conv2D)              (None, 16, 16, 32)        864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 16, 16, 32)       128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 16, 16, 32)        0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 16, 16, 32)       288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 16, 16, 32)       128       
 ation)                                         

In [22]:
base_model.trainable = True		# resnet 모델 학습동결을 해제한다
# for idx, i in enumerate(resnet50.layers):	# 143층부터의 학습은 해제상태로 두고, 
#   i.trainable = bool_arr[idx]				# 이전까지의 학습은 동결한다.

idx = 0 
for i in base_model.layers[:]:	              # 13 Blocks in MobileNet
  if i.name[7:9] == '_1' and idx == 4:
    print(idx, 'layer :  ', i.name)
  elif i.name[7:9] == '_1' and idx == 7:
    print(idx, 'layer :  ', i.name)
  elif i.name[7:10] == '_1_':
    print(idx, 'layer :  ', i.name)           # 1st Block
  elif i.name[7:9] == '_2':
    print(idx, 'layer :  ', i.name)           # 2nd Block
  elif i.name[7:9] == '_3':
    print(idx, 'layer :  ', i.name)           # 3rd Block
  elif i.name[7:9] == '_4':
    print(idx, 'layer :  ', i.name)           # 4th Block
  elif i.name[7:9] == '_5':
    print(idx, 'layer :  ', i.name)           # 5th Block
  elif i.name[7:9] == '_6':
    print(idx, 'layer :  ', i.name)           # 6th Block
  elif i.name[7:9] == '_7':
    print(idx, 'layer :  ', i.name)           # 7th Block
  elif i.name[7:9] == '_8':
    print(idx, 'layer :  ', i.name)           # 8th Block
  elif i.name[7:9] == '_9':
    print(idx, 'layer :  ', i.name)           # 9th Block
  elif i.name[7:10] == '_10':
    print(idx, 'layer :  ', i.name)           # 10th Block
  elif i.name[7:10] == '_11':
    print(idx, 'layer :  ', i.name)           # 11th Block
  elif i.name[7:10] == '_12':
    print(idx, 'layer :  ', i.name)           # 12th Block
  elif i.name[7:10] == '_13':
    print(idx, 'layer :  ', i.name)           # 13th Block
    
  idx += 1

4 layer :   conv_dw_1
5 layer :   conv_dw_1_bn
6 layer :   conv_dw_1_relu
7 layer :   conv_pw_1
8 layer :   conv_pw_1_bn
9 layer :   conv_pw_1_relu
11 layer :   conv_dw_2
12 layer :   conv_dw_2_bn
13 layer :   conv_dw_2_relu
14 layer :   conv_pw_2
15 layer :   conv_pw_2_bn
16 layer :   conv_pw_2_relu
17 layer :   conv_dw_3
18 layer :   conv_dw_3_bn
19 layer :   conv_dw_3_relu
20 layer :   conv_pw_3
21 layer :   conv_pw_3_bn
22 layer :   conv_pw_3_relu
24 layer :   conv_dw_4
25 layer :   conv_dw_4_bn
26 layer :   conv_dw_4_relu
27 layer :   conv_pw_4
28 layer :   conv_pw_4_bn
29 layer :   conv_pw_4_relu
30 layer :   conv_dw_5
31 layer :   conv_dw_5_bn
32 layer :   conv_dw_5_relu
33 layer :   conv_pw_5
34 layer :   conv_pw_5_bn
35 layer :   conv_pw_5_relu
37 layer :   conv_dw_6
38 layer :   conv_dw_6_bn
39 layer :   conv_dw_6_relu
40 layer :   conv_pw_6
41 layer :   conv_pw_6_bn
42 layer :   conv_pw_6_relu
43 layer :   conv_dw_7
44 layer :   conv_dw_7_bn
45 layer :   conv_dw_7_relu
46 la

In [23]:
size = 4 + 13         # 17
sample_arr = [True, False]
bool_arr = np.random.choice(sample_arr, size=size)
print(bool_arr)

[ True False False  True  True  True  True  True False  True  True False
  True False  True  True  True]


In [24]:
for idx, i in enumerate(base_model.layers):
    if i.name[7:9] == '_1' and idx == 4:
        if bool_arr[4] == True:
            i.trainable = True
        elif bool_arr[4] == False:
            i.trainable = False
        # print(idx, 'layer : ', i.name, i.trainable)
    elif i.name[7:9] == '_1' and idx == 7:
        if bool_arr[4] == True:
            i.trainable = True
        elif bool_arr[4] == False:
            i.trainable = False
        # print(idx, 'layer : ', i.name, i.trainable)
    elif i.name[7:10] == '_1_':
        if bool_arr[4] == True:
            i.trainable = True
        elif bool_arr[4] == False:
            i.trainable = False
        # print(idx, 'layer : ', i.name, i.trainable)

    elif i.name[7:9] == '_2':
        if bool_arr[5] == True:
            i.trainable = True
        elif bool_arr[5] == False:
            i.trainable = False
        # print(idx, 'layer : ', i.name, i.trainable)
    elif i.name[7:9] == '_3':
        if bool_arr[6] == True:
            i.trainable = True
        elif bool_arr[6] == False:
            i.trainable = False
        # print(idx, 'layer : ', i.name, i.trainable)
    elif i.name[7:9] == '_4':
        if bool_arr[7] == True:
            i.trainable = True
        elif bool_arr[7] == False:
            i.trainable = False
        # print(idx, 'layer : ', i.name, i.trainable)
    elif i.name[7:9] == '_5':
        if bool_arr[8] == True:
            i.trainable = True
        elif bool_arr[8] == False:
            i.trainable = False
        # print(idx, 'layer : ', i.name, i.trainable)
    elif i.name[7:9] == '_6':
        if bool_arr[9] == True:
            i.trainable = True
        elif bool_arr[9] == False:
            i.trainable = False
        # print(idx, 'layer : ', i.name, i.trainable)
    elif i.name[7:9] == '_7':
        if bool_arr[10] == True:
            i.trainable = True
        elif bool_arr[10] == False:
            i.trainable = False
        # print(idx, 'layer : ', i.name, i.trainable)                
    elif i.name[7:9] == '_8':
        if bool_arr[11] == True:
            i.trainable = True
        elif bool_arr[11] == False:
            i.trainable = False
        # print(idx, 'layer : ', i.name, i.trainable)
    elif i.name[7:9] == '_9':
        if bool_arr[12] == True:
            i.trainable = True
        elif bool_arr[12] == False:
            i.trainable = False
        # print(idx, 'layer : ', i.name, i.trainable)
    elif i.name[7:10] == '_10':
        if bool_arr[13] == True:
            i.trainable = True
        elif bool_arr[13] == False:
            i.trainable = False
        # print(idx, 'layer : ', i.name, i.trainable)
    elif i.name[7:10] == '_11':
        if bool_arr[14] == True:
            i.trainable = True
        elif bool_arr[14] == False:
            i.trainable = False
        # print(idx, 'layer : ', i.name, i.trainable)
    elif i.name[7:10] == '_12':
        if bool_arr[15] == True:
            i.trainable = True
        elif bool_arr[15] == False:
            i.trainable = False
        # print(idx, 'layer : ', i.name, i.trainable)
    elif i.name[7:10] == '_13':
        if bool_arr[16] == True:
            i.trainable = True
        elif bool_arr[16] == False:
            i.trainable = False
        # print(idx, 'layer : ', i.name, i.trainable)

In [25]:
for idx, i in enumerate(base_model.layers):
    print(idx, 'layer : ', i.name, '->' ,i.trainable)

0 layer :  input_1 -> True
1 layer :  conv1 -> True
2 layer :  conv1_bn -> True
3 layer :  conv1_relu -> True
4 layer :  conv_dw_1 -> True
5 layer :  conv_dw_1_bn -> True
6 layer :  conv_dw_1_relu -> True
7 layer :  conv_pw_1 -> True
8 layer :  conv_pw_1_bn -> True
9 layer :  conv_pw_1_relu -> True
10 layer :  conv_pad_2 -> True
11 layer :  conv_dw_2 -> True
12 layer :  conv_dw_2_bn -> True
13 layer :  conv_dw_2_relu -> True
14 layer :  conv_pw_2 -> True
15 layer :  conv_pw_2_bn -> True
16 layer :  conv_pw_2_relu -> True
17 layer :  conv_dw_3 -> True
18 layer :  conv_dw_3_bn -> True
19 layer :  conv_dw_3_relu -> True
20 layer :  conv_pw_3 -> True
21 layer :  conv_pw_3_bn -> True
22 layer :  conv_pw_3_relu -> True
23 layer :  conv_pad_4 -> True
24 layer :  conv_dw_4 -> True
25 layer :  conv_dw_4_bn -> True
26 layer :  conv_dw_4_relu -> True
27 layer :  conv_pw_4 -> True
28 layer :  conv_pw_4_bn -> True
29 layer :  conv_pw_4_relu -> True
30 layer :  conv_dw_5 -> False
31 layer :  conv_dw

In [9]:
IMG_SHAPE = (32, 32, 3)
base_model = MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
len(base_model.layers)

9420800/9406464 [==============================] - 2s 0us/step


154

In [10]:
IMG_SHAPE = (32, 32, 3)
base_model = VGG16(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
len(base_model.layers)

19

In [9]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [10]:
len(base_model.layers)

19

In [11]:
np.random.seed(0)

sample_arr = [True, False]
bool_arr = np.random.choice(sample_arr, size=len(base_model.layers))
print('Numpy Array: ')
print(bool_arr)
print(len(bool_arr))

Numpy Array: 
[ True False False  True False False False False False False False  True
  True False  True  True  True  True  True]
19


In [12]:
IMG_SHAPE = (32, 32, 3)
base_model = tf.keras.applications.resnet.ResNet50(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
len(base_model.layers)

175

In [11]:
IMG_SHAPE = (32, 32, 3)
base_model = tf.keras.applications.resnet.ResNet101(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
len(base_model.layers)

171458560/171446536 [==============================] - 21s 0us/step


345

In [14]:
IMG_SHAPE = (32, 32, 3)
base_model = tf.keras.applications.resnet.ResNet152(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
len(base_model.layers)

234708992/234698864 [==============================] - 23s 0us/step


515

In [5]:
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.applications.efficientnet import EfficientNetB1
from tensorflow.keras.applications.efficientnet import EfficientNetB2
from tensorflow.keras.applications.efficientnet import EfficientNetB3
from tensorflow.keras.applications.efficientnet import EfficientNetB4
from tensorflow.keras.applications.efficientnet import EfficientNetB5
from tensorflow.keras.applications.efficientnet import EfficientNetB6
from tensorflow.keras.applications.efficientnet import EfficientNetB7

In [6]:
IMG_SHAPE = (32, 32, 3)
base_model = EfficientNetB0(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
len(base_model.layers)

16719872/16705208 [==============================] - 2s 0us/step


237

In [7]:
IMG_SHAPE = (32, 32, 3)
base_model = EfficientNetB1(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
len(base_model.layers)

27033600/27018416 [==============================] - 2s 0us/step


339